In [23]:
!pip install pandas numpy

In [1]:
import os

CodeDir = os.getcwd()
BaseDir = os.path.split(CodeDir)[0]
DataDir = os.path.join(os.path.split(CodeDir)[0], 'Data')

print("Code Directory:",CodeDir)
print("Data Directory:",DataDir)

print("Contents of the Data Directory:",os.listdir(DataDir))
print("Contents of the Data Directory:",os.listdir(CodeDir))

Code Directory: /home/slayer/Git/VindiataInternshipTest/Code
Data Directory: /home/slayer/Git/VindiataInternshipTest/Data
Contents of the Data Directory: ['AircraftType.csv', 'CityPairs.csv', 'Operations.csv']
Contents of the Data Directory: ['Untitled.ipynb', 'DataManipulation.ipynb', '.ipynb_checkpoints']


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import re
import numpy as np

In [4]:
def csv_files(s):
    return '.csv' in s

csv_files = list(filter(csv_files,os.listdir(DataDir)))

In [5]:
csv_files

['AircraftType.csv', 'CityPairs.csv', 'Operations.csv']

In [6]:
AircraftType = pd.read_csv(os.path.join(DataDir,csv_files[0]))
CityPairs = pd.read_csv(os.path.join(DataDir,csv_files[1]))
Operations = pd.read_csv(os.path.join(DataDir,csv_files[2]))

In [7]:
column_list = list(Operations)
Operations["SumOfHours"] = Operations[column_list[2:]].sum(axis = 1)

## What is the total cost by aircraft type for the year?

In [8]:
Operations['Cost'] = np.nan
for i in range(len(Operations.index)):
    Operations['Cost'][i] = int(AircraftType.loc[AircraftType['Aircraft Type'] == Operations['Aircraft Type'][i].strip(), 'Costs per flight hour'].values[0].strip().replace(',',''))
    
Operations['TotalCost'] = Operations['Cost'] * Operations['SumOfHours']
Operations.head()

,Aircraft Name,Aircraft Type,Jan-14,Feb-14,Mar-14,Apr-14,May-14,Jun-14,Jul-14,Aug-14,Sep-14,Oct-14,Nov-14,Dec-14,SumOfHours,Cost,TotalCost
0,A,A320,250,251,248,253,243,245,248,246,256,256,251,252,2999,5000.0,14995000.0
1,B,A330,300,301,298,303,293,295,298,296,306,306,301,302,3599,7500.0,26992500.0
2,C,B737,245,246,243,248,238,240,243,241,251,251,246,247,2939,5100.0,14988900.0
3,D,B747,310,311,308,313,303,305,308,306,316,316,311,312,3719,12000.0,44628000.0
4,E,Q400,200,201,198,203,193,195,198,196,206,206,201,202,2399,3500.0,8396500.0


In [9]:
print("Total Cost Of Operations:",Operations['TotalCost'].sum(axis = 0))

Total Cost Of Operations: 490225600.0


## Which aircraft type has the lowest cost per seat per km flown? 

In [10]:
AircraftType['Costs per flight hour'] = pd.to_numeric(AircraftType['Costs per flight hour'].transform(lambda x:str(x).strip().replace(',','')))
AircraftType['lcps'] = AircraftType['Costs per flight hour']/(AircraftType['Number of Seats']*AircraftType['Costs per flight hour'])

In [11]:
m = min(AircraftType['lcps'])

print("Lowest cost per seat per km flown:", m)
print("Aircraft Type :",AircraftType.loc[AircraftType['lcps'] == m, 'Aircraft Type'].values[0])

Lowest cost per seat per km flown: 0.002857142857142857
Aircraft Type : B747


## Airline B is a startup airline and is considering flying on the city pairs as shown in tab "City Pairs". They would like to know:
### Which aircraft types are best suited for their operation?

In [19]:
CityPairs['Distance (km)'] = pd.to_numeric(CityPairs['Distance (km)'].transform(lambda x: str(x).strip().replace(',','')))
AircraftType['Range (Km)'] = pd.to_numeric(AircraftType['Range (Km)'].transform(lambda x: str(x).strip().replace(',','')))

In [21]:
from math import ceil
def minCost(dist, n):
    t = AircraftType.loc[AircraftType['Range (Km)'] >= dist]
    t['cost per trip'] = (dist/t['Ave. Speed (km/h)'])*t['Costs per flight hour']
    t['no trips'] = n/t['Number of Seats'].apply(np.ceil).astype(int)
    t['Total Cost'] = t['cost per trip']*t['no trips']
    #print(t)
    m = min(t['Total Cost'])
    r = t.loc[t['Total Cost'] == m , 'Aircraft Type']
    del m,t
    return r.values[0]
    

In [22]:
CityPairs['AircraftName'] = np.vectorize(minCost)(CityPairs['Distance (km)'], CityPairs['Pass. Demand \n(per day)'])
CityPairs

,Origin City,Desitnation City,Pass. Demand \n(per day),Distance (km),AircraftName
0,AA,BB,420,3000,A330
1,BB,CC,450,6500,A330
2,CC,AA,300,400,A330
3,AA,DD,300,1000,A330
